<a href="https://colab.research.google.com/github/grupo274/pre-projeto-i2a2/blob/main/I2A2_firstAgent_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load API key from file

In [6]:
#file_path = 'openrouter_api_key.txt'
file_path = 'devstral_small_api_key.txt'

try:
  with open(file_path, 'r') as file:
    API_KEY = file.read()
    print(f"The file was successfully loaded.")
except FileNotFoundError:
  print(f"Error: The file '{file_path}' was not found.")
except Exception as e:
  print(f"An error occurred: {e}")

The file was successfully loaded.


# Functions that implement the tools

In [12]:
import zipfile
import pandas as pd
import io, json

# Define the functions that implement the tools
def unzip_file(filename):
  """
  Unzips a given file.

  Args:
    filename: The name of the ZIP file to unzip.

  Returns:
    A list of the names of the extracted files, or an error message.
  """
  if filename.endswith('.zip'):
    try:
      with zipfile.ZipFile(filename, 'r') as zip_ref:
        print("unzip_file function was activated.")
        zip_ref.extractall()
        return zip_ref.namelist()
    except zipfile.BadZipFile:
      return f"Error: '{filename}' is not a valid zip file."
    except FileNotFoundError:
      return f"Error: The file '{filename}' was not found."
    except Exception as e:
      return f"An error occurred while unzipping '{filename}': {e}"


def read_csv(filename: str):
    """Reads a CSV file and returns its content."""
    if filename.endswith('.csv'):
      try:
        with open(filename, 'r') as f:
          print("read_csv function was activated.") ##
          content = f.read()
          return content
      except FileNotFoundError:
        return f"Error: File '{filename}' not found."
      except Exception as e:
        return f"An error occurred while reading '{filename}': {e}"


def analyze_data(data: str, analysis_type: str, column: str, top_n: int = None):
  """
  Analyzes a dataframe based on the provided data, analysis type, and column.

  Args:
    data: The data to analyze (string format, e.g., content of a CSV).
    analysis_type: The type of analysis to perform (e.g., 'most frequent item', 'total volume', 'average').
    column: The column to perform the analysis on.

  Returns:
    dict: The result of the analysis or an error message.
  """
  try:
    df = pd.read_csv(io.StringIO(data))

    if column not in df.columns:
      return {
          "status": "error",
          "message": f"Column '{column}' not found in the data.",
          "available_colums": list(df.columns)
      }
    result = None

    if analysis_type == 'most frequent item':
      if pd.api.types.is_numeric_dtype(df[column]):
        return {
            "status": "error",
            "message": f"Error: 'most frequent item' analysis is typically for categorical data, but column '{column}' is numeric."
        }
      result = df[column].mode().tolist()

    elif analysis_type == 'total volume':
      if not pd.api.types.is_numeric_dtype(df[column]):
        return {
            "status": "error",
            "message": f"Error: 'total volume' analysis requires a numeric column, but column '{column}' is not."
        }
      result = df[column].sum()

    elif analysis_type == 'average':
      if not pd.api.types.is_numeric_dtype(df[column]):
        return {
                    "status": "error",
                    "message": f"'average' requer coluna numérica. A coluna '{column}' não é."
                }
      result = df[column].mean()

    elif analysis_type == 'top items':
      if not pd.api.types.is_numeric_dtype(df[column]):
        return {
            "status": "error",
            "message": f"'top items' requer coluna numérica para somatório. A coluna '{column}' não é."
            }
      if not top_n or top_n <= 0:
        return {
            "status": "error",
            "message": "Você deve fornecer um valor positivo para 'top_n'."
            }
      # Assuming there is a categorical column (e.g. 'product') and this is the group key
      group_column = [col for col in df.columns if col != column][0]
      grouped = df.groupby(group_column)[column].sum().nlargest(top_n)
      result = grouped.to_dict()

    # Add more analysis types as needed

    else:
      return {
          "status": "error",
          "message": f"Error: Analysis type '{analysis_type}' is not supported.",
          "supported_analyses": ['most frequent item', 'total volume', 'average']
      }

    return {
        "status": "success",
        "analysis_type": analysis_type,
        "column": column,
        "result": result
      }

  except Exception as e:
    return {
            "status": "error",
            "message": f"Ocorreu um erro ao analisar os dados: {e}"
        }

# Add more tool functions as needed

# Setting the tools

In [13]:
available_functions = {
    "unzip_file": unzip_file,
    "read_csv": read_csv,
    "analyze_data": analyze_data,
    # Add more functions if needed
}


tools = [
    {
        "type": "function",
        "function": {
            "name": "unzip_file",
            "description": "Descompacta um arquivo zip e lista os arquivos extraídos.",
            "parameters": {
                "type": "object",
                "properties": {
                    "zip_path": {
                        "type": "string",
                        "description": "Caminho para o arquivo zip a ser descompactado"
                    },
                },
                "required": ["zip_path"]
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "read_csv",
            "description": "Lê um arquivo CSV e retorna as primeiras linhas.",
            "parameters": {
                "type": "object",
                "properties": {
                    "file_path": {
                        "type": "string",
                        "description": "Caminho para o arquivo CSV"
                    },
                },
                "required": ["file_path"]
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "analyze_data",
            "description": "Executa uma análise estatística descritiva em um arquivo CSV.",
            "parameters": {
                "type": "object",
                "properties": {
                    "file_path": {
                        "type": "string",
                        "description": "Caminho para o arquivo CSV"
                    },
                },
                "required": ["file_path"]
            },
        },
    },
    # Add more tools if needed
]

# User prompt

In [14]:
user_prompt = "Unzip o arquivo 202401_NFs.zip e analise o arquivo 202401_NFs_Itens.csv para identificar qual item teve maior volume entregue (em quantidade)?" #@param {type:"string"}

# AI Agent

In [15]:
from openai import OpenAI
import json
import pandas as pd
import zipfile
import os

# Connect client
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key = API_KEY   #<OPENROUTER_API_KEY>
)


# Add context about the available file and request tool usage explicitly
initial_message_content = f"Você deve usar as ferramentas disponíveis para responder à seguinte pergunta: {user_prompt}."

messages = [
    {"role": "user",
     "content": initial_message_content
     }
    ]


while True:
  completion = client.chat.completions.create(
      model="mistralai/devstral-small:free",
      messages=messages,
      tools=tools
  )

  if not completion.choices or completion.choices[0].message is None:
    print("Nenhuma resposta do modelo.")
    break

  response_message = completion.choices[0].message
  tool_calls = getattr(response_message, "tool_calls", None)

  if tool_calls:
    messages.append(response_message)

    for tool_call in tool_calls:
      function_name = tool_call.function.name
      function_to_call = available_functions.get(function_name)

      if not function_to_call:
        raise ValueError(f"Função '{function_name}' não encontrada.")

      try:
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(**function_args)
      except Exception as e:
        function_response = f"Erro ao executar a função '{function_name}': {e}"

      messages.append({
          "tool_call_id": tool_call.id,
          "role": "tool",
          "name": function_name,
          "content": str(function_response)
      })

      # Se arquivos CSV foram extraídos, aplica leitura e análise a todos
      if "unzip_file" in [tc.function.name for tc in tool_calls]:
        extracted_files = function_response if isinstance(function_response, list) else []
        for file in extracted_files:
          if file.endswith(".csv"):
            for func in [read_csv, analyze_data]:
              try:
                response = func(file)
                print(f"Resultado de {func.__name__} para {file}:\n{response}\n")
              except Exception as e:
                print(f"Erro em {func.__name__} para {file}: {e}")
  else:
    print(response_message.content or "Modelo não respondeu após as chamadas de ferramenta.")
    break

Claro! Vou seguir os seguintes passos para responder à sua pergunta:

1. Descompactar o arquivo `202401_NFs.zip`.
2. Ler o arquivo `202401_NFs_Itens.csv`.
3. Identificar qual item teve o maior volume entregue (em quantidade).

Vamos começar!


# Perguntas
1- Considere o arquivo 202401_NFs_Itens.csv, qual é o assunto que ele possui?

R: O arquivo **202401_NFs_Itens.csv** refere-se a dados relacionados com **itens de notas fiscais** (NFs). Pela nomenclatura, podemos inferir que se trata de informações detalhadas sobre os produtos ou serviços associados a notas fiscais emitidas em **janeiro de 2024**. Isso inclui dados como:

1. **Códigos e descrições de produtos/serviços**  
2. **Quantidades**  
3. **Valores unitários e totais**  
4. **Impostos associados** (ICMS, IPI, etc.)  
5. **Chaves de acesso das notas fiscais**  
6. **Identificadores de clientes e fornecedores**  

O assunto principal é a **gestão fiscal e contábil de transações comerciais**, especificamente vinculado à estrutura de itens que compõem as notas fiscais eletrônicas (NF-e ou NFC-e). Esses dados são críticos para:  
- Controle de estoque,  
- Cálculo de tributos,  
- Auditoria fiscal,  
- Análise financeira.  

📌 **Contexto adicional**:  
O formato `.csv` indica que é uma base estruturada, provavelmente gerada por sistemas ERP (como SAP, Totvs) ou emissoras de notas fiscais (ex.: SEFAZ). O prefixo "202401" sugere periodicidade mensal (janeiro/2024), comum em relatórios fiscais e arquivos de SPED (Sistema Público de Escrituração Digital).



2- Considere o arquivo 202401_NFs_Itens.csv, qual é o fornecedor que teve maior montante recebido?

R:


3- Qual item teve maior volume entregue (em quantidade)?

R: Para identificar o item com maior volume entregue em quantidade no arquivo **202401_NFs_Itens.csv**, siga os passos abaixo:

### Passo a Passo:
1. **Leia o arquivo CSV**:  
   Carregue o conteúdo do arquivo usando uma ferramenta que permita manipulação de dados (ex: Python, Excel, R).

2. **Agrupe os itens por código e some as quantidades**:  
   - Agrupe todas as linhas pelo campo `CodItem`.  
   - Some os valores da coluna `Quant` (quantidade entregue) para cada grupo.

3. **Identifique o item com a maior soma**:  
   Encontre o registro com o valor máximo na soma das quantidades.

### Solução (usando Python com pandas):
```python
import pandas as pd

# Carregar o arquivo CSV
df = pd.read_csv("202401_NFs_Itens.csv", delimiter=';')  # Verifique o delimitador

# Agrupar por 'CodItem' e somar as quantidades
volume_por_item = df.groupby("CodItem")["Quant"].sum().reset_index()

# Encontrar o item com maior volume
item_maior_volume = volume_por_item.loc[volume_por_item["Quant"].idxmax()]

print("Item com maior volume entregue:")
print(f"Código: {item_maior_volume['CodItem']}")
print(f"Total Entregue: {item_maior_volume['Quant']}")
```

### Explicação:
- **groupby("CodItem")**: Agrupa as linhas pelo código do item.
- **["Quant"].sum()**: Soma a coluna `Quant` (quantidade) para cada grupo.
- **idxmax()**: Localiza o índice do valor máximo na coluna `Quant`.

### Resultado:
O código acima irá gerar uma saída similar a:
```
Item com maior volume entregue:
Código: [Código do Item]
Total Entregue: [Valor Máximo]
```
Substitua `[Código do Item]` e `[Valor Máximo]` pelo resultado real encontrado no arquivo.

### Observação importante:
Verifique se o arquivo utiliza **virgula**, **ponto e vírgula** ou outro delimitador. Ajuste o parâmetro `delimiter` em `pd.read_csv()` conforme necessário. Exemplo comum em dados brasileiros: `delimiter=';'`.

Para confirmar o resultado manualmente, você também pode:
1. Abrir o CSV em Excel.
2. Inserir uma tabela dinâmica:
   - Linhas: `CodItem`
   - Valores: Soma de `Quant`
3. Ordenar a coluna de soma em ordem decrescente.


4- Qual é o maior consumidor final e o número de itens comprados?

R: Para determinar o maior consumidor final (pessoa física que compra para uso próprio) e o número total de itens comprados, siga as etapas no código Python abaixo. O arquivo CSV `202401_NFs_Itens.csv` deve conter uma coluna que identifique o tipo de cliente (ex.: `Tipo_Cliente` com valor `'Consumidor Final'`), um identificador único para o cliente (ex.: `CPF`) e a quantidade de itens por registro.

```python
import pandas as pd

# Carregar o arquivo CSV (ajuste sep/encoding conforme necessário)
df = pd.read_csv('202401_NFs_Itens.csv', sep=';', encoding='latin1')

# Verificar colunas (para depuração)
# print(df.columns.tolist())

# Pressupostos sobre as colunas:
#   - Coluna de tipo de cliente: 'Tipo_Cliente'
#   - Coluna de identificação do cliente: 'CPF'
#   - Coluna de quantidade: 'Quantidade'

# Filtrar apenas consumidores finais
df_consumidor_final = df[df['Tipo_Cliente'] == 'Consumidor Final']

# Agrupar por CPF e somar as quantidades
compras_por_cliente = df_consumidor_final.groupby('CPF')['Quantidade'].sum()

# Identificar o maior consumidor
maior_consumidor_id = compras_por_cliente.idxmax()
total_itens = compras_por_cliente.max()

print(f"Maior consumidor final: CPF {maior_consumidor_id}")
print(f"Total de itens comprados: {total_itens}")
```

### Explicação:
1. **Filtragem de Consumidores Finais**: Apenas registros marcados como `'Consumidor Final'` são considerados.
2. **Agregação**: Itens são somados por cliente (usando `CPF` como identificador).
3. **Identificação do Maior Consumidor**: O cliente com a maior soma de itens é extraído.

### Observações:
- **Colunas do arquivo**: Se as colunas no CSV tiverem nomes diferentes (ex.: `CNPJ_CPF`, `Qtde`), ajuste os nomes no código.
- **Formato do arquivo**: Se o CSV usar separadores diferentes (ex.: vírgula), ajuste `sep` (ex.: `sep=','`).
- **Resultado**: A saída será o CPF do maior consumidor e o total de itens comprados por ele. Se houver empate, retornará o primeiro cliente com a quantidade máxima.

### Execução:
Execute o código com o arquivo no mesmo diretório. Se precisar de detalhes da estrutura do arquivo, descomente `print(df.columns.tolist())`.

Resultado esperado (exemplo):
```
Maior consumidor final: CPF 123.456.789-00
Total de itens comprados: 150
```